# 스키마 정의 후 Vector Store 생성 

---

---

## Setting
 - Auto Reload
 - path for utils

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [22]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [23]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Embedding 모델 로딩

## Embedding Model 선택

In [28]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

In [29]:
llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(
        model_name="Titan-Embeddings-G1"
    )
)
dimension = 1536

llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7fc360e6fe80>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

## 3.데이터 준비 

###  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [ ]:
import pandas as pd
pd.options.display.max_rows = 20

In [32]:
data_file_path = "./data/shinhan/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


### 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [33]:
os.makedirs("data", exist_ok=True)

In [34]:
def preprocess_data(df):

    ldf = df.copy()
    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    df_index.to_csv("./data/shinhan/fsi_smart_faq_ko_processed.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [37]:
import time
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter

In [38]:
loader = CSVLoader(
    file_path="./data/shinhan/fsi_smart_faq_ko_processed.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)

documents_fsi = loader.load()

In [42]:
documents_fsi[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1708062603.3835008})

In [40]:
def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):

        #print ("previous:", doc)
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        split_content = doc.page_content.split("type: ")
        content = split_content[0]
        metadata = split_content[1]
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        doc.metadata['timestamp'] = time.time()

create_metadata(documents_fsi)

In [41]:
print (len(documents_fsi))
print (documents_fsi[0])

92
page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n' metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1708062603.3835008}


### Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. <BR>
또한 임베딩 모델에는 입력 토큰 길이는 KoSimCSERobert는 512개, titanEmbedding(8,912개) 토큰으로 제한되어 있습니다. <BR>
이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 500자가 겹치는 약 92자의 청크를 생성합니다.

In [62]:
chunk_size = 2048
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

chunk_docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


In [63]:
chunk_docs[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1708062603.3835008})

## 4. Index 생성

### Index 이름 결정

In [152]:
index_name = <your index>

## Index 스키마 정의

In [65]:
index_body = {
    'settings': {
        'analysis': {
            'analyzer': {
                'my_analyzer': {
                         'char_filter':['html_strip'],
                    'tokenizer': 'nori',
                    'filter': [
                        'nori_number',
                        'lowercase',
                        'trim',
                        'my_nori_part_of_speech'
                    ],
                    'type': 'custom'
                }
            },
            'tokenizer': {
                'nori': {
                    'decompound_mode': 'mixed',
                    'discard_punctuation': 'true',
                    'type': 'nori_tokenizer'
                }
            },
            "filter": {
                "my_nori_part_of_speech": {
                    "type": "nori_part_of_speech",
                    "stoptags": [
                        "E", "IC", "J", "MAG", "MAJ",
                        "MM", "SP", "SSC", "SSO", "SC",
                        "SE", "XPN", "XSA", "XSN", "XSV",
                        "UNA", "NA", "VSV"
                    ]
                }
            }
        },
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                    'source' : {'type': 'keyword'},
                    'row': {'type': 'long'},
                    'type': {'type': 'keyword'},
                    'timestamp': {'type': 'float'},
                }
            },
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': f"{dimension}" # Replace with your vector dimension
            }
        }
    }
}


## 5. LangChain OpenSearch VectorStore 생성 
### 선수 조건


### 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [66]:
import boto3
from ssm import parameter_store

In [67]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [68]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [69]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### OpenSearch Client 생성

In [70]:
from opensearch import opensearch_utils

In [71]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [72]:
from utils.opensearch import opensearch_utils

In [73]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=dongjin-index, exists=True

Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'dongjin-index'}
Index is created
{'dongjin-index': {'aliases': {},
                   'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                           'source': {'type': 'keyword'},
                                                                           'timestamp': {'type': 'float'},
                                                                           'type': {'type': 'keyword'}}},
                                               'text': {'analyzer': 'my_analyzer',
                                                        'type': 'text'},
                                               'vector_field': {'dimension': 1536,
                                                                'type': 'knn_vector'}}},
                   'settings'

### 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [74]:
from langchain.vectorstores import OpenSearchVectorSearch

In [75]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

### OpenSearch 에 문서 삽입

In [76]:
%%time

vector_db.add_documents(
    documents = chunk_docs, 
    vector_field = "vector_field",
    bulk_size = 1000000
)


CPU times: user 312 ms, sys: 11.4 ms, total: 323 ms
Wall time: 21.7 s


['f9840209-311a-41b8-b20e-f822eacc3ca7',
 '5803cdac-3c78-4186-b452-7e158bffb298',
 '9d3ccd09-5e54-4af4-a2bd-bbe7366cbc49',
 '5db89ef9-2c92-40d4-a3f1-dfa4bce7b5f1',
 'd7bc901f-8108-410f-a887-052b3d9309cf',
 '56cf9b0c-1f48-4fd0-8369-7a9d919ad265',
 '4b44d38c-2ccd-43e1-95c0-217e13dcda57',
 '2d425714-b74d-41b8-b608-be92844e9eea',
 'ff129b8d-8636-4e99-976b-cb7f852e5ad4',
 '6822cec4-1b94-4c57-9ba7-025befe3bdb6',
 '6d669c5e-d9c4-4af9-b7a3-d1a8e02597ec',
 '8b4d973d-02b6-43ee-81ea-05d278145631',
 'd8833740-319f-46cc-b3c6-ce80fe9a82e5',
 '2c929869-7c8c-4f95-bdaa-cec78f0da24b',
 'a33a9624-7b63-4e10-8931-18cfa078189d',
 '0e878e45-5b34-4b59-a8e7-d86fd98fd338',
 '6c58a96d-09d9-48af-aef2-51160c3ce357',
 '72fe3f44-0279-436d-ba08-6ae8d6014ae8',
 '67f3bd8d-6b56-4bfa-8027-9e18d2ea2765',
 '19771180-5037-4164-8fb4-5c28c5876c3c',
 'bf61ff4a-cb64-4a00-89be-06ae176aa277',
 '9ecea392-6514-4a1f-a767-4a1136d08a15',
 '95ab8acf-e4f3-4e05-9db7-96bdbd1895aa',
 '3f1abd86-598a-4f47-a6e8-169d31ea729b',
 '709af891-56f9-

## 6. 검색 및 질의 응답 테스트

In [132]:
from rag import retriever_utils
from rag import show_context_used
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### LLM 선택

In [129]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7fc360e6fe80>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fc349835de0>])

### QA prompt

In [131]:
prompt_template = """

Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [133]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

### Semantic 검색

In [146]:
query = "중지된 경우 이체"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [147]:
similar_docs_semantic = retriever_utils.get_semantic_similar_docs(
    index_name=index_name,
    os_client=os_client,
    llm_emb=llm_emb,
    query=query,
    k=5,
    boolean_filter=search_filter,
    hybrid=False
)
show_context_used(similar_docs_semantic)

-----------------------------------------------
1. Chunk: 402 Characters
-----------------------------------------------
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는
이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이
1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③
최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)
metadata:
 {'source': '신한은행', 'row': 39, 'type': '인터넷뱅킹', 'timestamp': 1708062603.3836446, 'id':
'3d0ce6d8-2527-46b5-ab57-8ba1b22a5d94'}
-----------------------------------------------
2. Chunk: 584 Characters
-----------------------------------------------
ask: 이체한도 증액은 어떻게 하나요?
Information: 영업점 방문하지 않아도 모바일뱅킹 신한 쏠(SOL) 에서 비대면실명 인증후 통합이체한도 증액 가능합니다. 통합이체한도는 이용하시는 보안매체에 따라,
보안등급별 최고 한도내에서 증액가능합니다.
OTP카드 창구 발급 시 5,000원(카드형 10,000원)의 수수료가 부과됩니다. 당행에서만 사용가능한 모바일OTP는 영업점 방문없이

In [148]:
answer = chain.run(
    input_documents=similar_docs_semantic,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)

ask: 이체한도 증액은 어떻게 하나요?
Information: 영업점 방문하지 않아도 모바일뱅킹 신한 쏠(SOL) 에서 비대면실명 인증후 통합이체한도 증액 가능합니다. 통합이체한도는 이용하시는 보안매체에 따라, 보안등급별 최고 한도내에서 증액가능합니다.
OTP카드 창구 발급 시 5,000원(카드형 10,000원)의 수수료가 부과됩니다. 당행에서만 사용가능한 모바일OTP는 영업점 방문없이 신한 쏠(SOL) 에서 무료로 발급가능합니다. 
※ 모바일OTP를 이용하시는 경우 쏠(SOL)에서 이체한도 증액은 1회 1천만원 이하, 1일 5천만원 이하로 증액 가능합니다. 
※ 만 14세 미만의 경우 법정대리인(부모/친권자/후견인)에 의해 영업점 방문하여 증액 가능하며 1일, 1회 1백만원 이하로 이체한도 제한됩니다. 
통합한도란 인터넷뱅킹

## Lexical 검색

In [149]:
query = "중지된 경우 이체"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [150]:
similar_docs_lexical = retriever_utils.get_lexical_similar_docs(
    index_name=index_name,
    os_client=os_client,
    query=query,
    k=5,
    filter=search_filter
)
show_context_used(similar_docs_lexical)

-----------------------------------------------
1. Chunk: 591 Characters
-----------------------------------------------
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를
원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는
연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→
장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니다. 쏠(SOL)의 경로는 위와
동일하며, 비대면 실명인증후 해제 가능합니다. 비대면실명인증 - 만 14세이상 가능 - 신분증확인 (주민등록증 또는 운전면허증 또는 여권) - 본인확인 (본인명의 계좌인증 또는
영상통화) 진행 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다
metadata:
 {'source': '신한은행', 'row': 40, 'type': '인터넷뱅킹', 'timestamp': 1708062603.3836489, 'id':
'cc6ee9b3-2acc-49d8-b904-b1e4f6868c5f'}
-----------------------------------------------
2. Chunk: 375 Characters
-----------------------------------------------
ask: 타행으로 자동이체를 신청하고 싶은데요
Inf

In [151]:
answer = chain.run(
    input_documents=similar_docs_lexical,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는 연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→ 장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니다. 쏠(SOL)의 경로는 위와 동일하며, 비대면 실명인증후 해제 가능합니다. 비대면실명인증 - 만 14세이상 가능 - 신분증확인 (주민등록증 또는 운전면허증 또는 여권) - 본인확인 (본인명의 계좌인증 또는 영상통화) 진행 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다

ask: 타행으로 자동이체를 신청하고 싶은데요
Information: 인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는 건당 300원 입니다. 자동이체를 신청하시려면 인터넷뱅킹에 로그인 하신후 [이체] > [자동이체] > [자동이체 등록] 메뉴를 선택하시면 타행으로의 자동이체 등록을 하실 수 있습니다. ※ 주의: 타은행 자동이체는 이체 지정일 전영업일에 고객님의 

## 하이브리드 검색

In [143]:
query = "중지된 경우 이체"

search_filter=[
    {"term": {"metadata.source": "신한은행"}},
    {"term": {"metadata.type": "인터넷뱅킹"}},
]

In [144]:
%%time
similar_docs_hybrid = retriever_utils.search_hybrid(
    query=query,
    k=5,
    index_name=index_name,
    os_client=os_client,
    filter=search_filter,
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[0.51, 0.49], # semantic, lexical
    async_mode=True,
    llm_emb=llm_emb,
    verbose=True
)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
similar_docs_semantic
##############################
[(Document(page_content='ask: 이체한도란 무엇인가요?\nInformation: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)', metadata={'source': '신한은행', 'row': 39, 'type': '인터넷뱅킹', 'timestamp': 1708062603.3836446, 'id':

In [145]:
answer = chain.run(
    input_documents=similar_docs_hybrid,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)

ask: 인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요
Information: 인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고용으로 단지 조회일자 및 출금 계좌번호 등으로 상세내역(이체 확인증 목록)을 조회하거나 그 내역을 인쇄만 가능합니다. 인터넷뱅킹 거래 상세내역(이체확인증 목록)및 입금증(이체확인증) 인쇄 방법은 다음과 같습니다. ① 인터넷뱅킹에 접속하여 로그인 ② 이체 → 즉시이체 → 이체결과조회 ③ 조회구분, 출금계좌번호, 조회일자등 상세내역을 조회 ④ 입금확인증목록을 인쇄하시려면 조회된 내역 왼쪽 상단에 보이는 [이체목록인쇄] 버튼을 클릭하면 해당 거래내역을 출력할 수 있으며, 입금확인증을 인쇄하시려면 해당 이체건의 오른쪽에 있는 [인쇄] 버튼을 클릭하시면 됩니다. 

In [126]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

answer = chain.run(
    input_documents=similar_docs_lexical,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다.(단, 인터넷뱅킹 출금계좌가 등록되어 있는 경우) [경로] ① 당행 고객정보에 등록되어있는 연락처로 ARS 추가인증 가능한 경우: - 인터넷뱅킹 로그인→ 사용자관리→ 인터넷뱅킹관리→ 장기미사용 정지해제 (또는) - 쏠(SOL) 로그인→ 전체메뉴→ 이체 → 이체관리→ 장기미사용정지 해제 ② ARS 추가인증이 불가한 경우 (해외체류 고객등) : - 인터넷뱅킹에서는 해제 어려우며, 쏠 (SOL)에서 해제 가능합니다. 쏠(SOL)의 경로는 위와 동일하며, 비대면 실명인증후 해제 가능합니다. 비대면실명인증 - 만 14세이상 가능 - 신분증확인 (주민등록증 또는 운전면허증 또는 여권) - 본인확인 (본인명의 계좌인증 또는 영상통화) 진행 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다

ask: 타행으로 자동이체를 신청하고 싶은데요
Information: 인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는 건당 300원 입니다. 자동이체를 신청하시려면 인터넷뱅킹에 로그인 하신후 [이체] > [자동이체] > [자동이체 등록] 메뉴를 선택하시면 타행으로의 자동이체 등록을 하실 수 있습니다. ※ 주의: 타은행 자동이체는 이체 지정일 전영업일에 고객님의

# A. Reference

- [Building a RAG AI with OpenSearch Serverless and LangChain](https://caylent.com/blog/building-a-rag-with-open-search-serverless-and-lang-chain)